In [1]:
%reload_ext autoreload
%autoreload 2

# %matplotlib widget
%gui qt

import time
import random
from copy import copy
from tqdm import tqdm
from pathlib import Path
import numpy as np
import numba as nb
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.width = 1000

from vrAnalysis import session
from vrAnalysis import functions
from vrAnalysis import analysis
from vrAnalysis import helpers
from vrAnalysis import fileManagement as fm
from vrAnalysis import database

from vrAnalysis.redgui import redCellGUI as rgui

vrdb = database.vrDatabase()

(CVXPY) Oct 18 03:12:05 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')


In [125]:
# Immediate next steps for programming work:
# - ROICaT index alignment tools

In [241]:
# Database Requirements: 
# ---------------------
# GUI: db manager
# - show full database -
# - show selected entries (like for example entries with imaging but no suite2p) -
#                --> It would be sweet to make this work where there are some defaults, but then there is also a "filter command" option which you type and goes as the input to the filtering
# - click on entry and do things:
#                --> open file explorer to that session
#                --> do suite2p
#                --> do red cell management
# - update table data? 
# ---------------------
# Operational Commands: 
# - Automatically do suite2p 
# - Check if registration was done before a suite2p update
# - Find way to determine if a suite2p file was updated after the folder was made (indicating that QC was done afterwards) -- so registration should be rerun for imaging data at least
#    -- check if "iscell" is the only thing that was done after registration. In this case, just update the iscell oneData
# ---------------------
# - Add new entry GUI!!!
# ---------------------
# - Check session folders to see if new session is present that isn't registered in the database? 
# ---------------------
# vrExperimentUpdates: 
# -- if edited iscell, will need to re-register the number of ROIs per plane (in "self.values") and reprocess imaging data!
# ---------------------
# vrExperimentRegistration
# -- if error, then note there was an error and append an explanation...!!!
# ---------------------

In [2]:
mouseName = 'CR_Hippocannula6'
dateString = '2022-09-05'
sessionid = '701'
# vrExperiment registration and preprocessing 
# vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
# vrReg.processTimeline()
# vrReg.processImaging()
# vrReg.processRedCells()
# vrReg.saveParams()
# print(vrReg.sessionPath())
# del vrReg

In [154]:
for idx, ses in enumerate(vrdb.iterSessions(imaging=True, vrRegistration=True)):
    print(idx)
    pcss = analysis.placeCellSingleSession(ses)
    pcss.plot_snake(normalize=10, withSave=True, withShow=False)
    pcss.plot_remap_snakes(normalize=10, withSave=True, withShow=False)

0
placeCellSingleSession is saving a snake_plot figure for session: ATL012/2023-01-18/701
Session ATL012/2023-01-18/701 only uses 1 environment, not plotting remap snakes
1
placeCellSingleSession is saving a snake_plot figure for session: ATL012/2023-01-20/702
Session ATL012/2023-01-20/702 only uses 1 environment, not plotting remap snakes
2
placeCellSingleSession is saving a snake_plot figure for session: ATL012/2023-01-23/701
Session ATL012/2023-01-23/701 only uses 1 environment, not plotting remap snakes
3
placeCellSingleSession is saving a snake_plot figure for session: ATL012/2023-01-24/703
Session ATL012/2023-01-24/703 only uses 1 environment, not plotting remap snakes
4
placeCellSingleSession is saving a snake_plot figure for session: ATL012/2023-01-25/701
Session ATL012/2023-01-25/701 only uses 1 environment, not plotting remap snakes
5
placeCellSingleSession is saving a snake_plot figure for session: ATL012/2023-01-26/702
Session ATL012/2023-01-26/702 only uses 1 environment, 

In [19]:
# I need to remove the silly variables from CR_Hippos on D:/

In [26]:
# ROICaT Alignment Tools
# Track Red Cell Consistency across days

In [145]:
ses.sessionPrint()

'ATL012/2023-02-27/701'

In [151]:
vrexp = ses
distStep = (1,5,2)
speedThreshold = 5

distedges, distcenter, roomLength = functions.getBinEdges(vrexp, distStep)
numPosition = len(distcenter)

trialStartSample = vrexp.loadone('trials.positionTracking')
behaveTimeStamps = vrexp.loadone('positionTracking.times')
behavePosition = vrexp.loadone('positionTracking.position')
lickSamples = vrexp.loadone('licksTracking.positionTracking') # list of behave sample for each lick

behavePositionBin = np.digitize(behavePosition,distedges)-1 # index of position bin for each sample
lickPositionBin = behavePositionBin[lickSamples] # index of position bin for each lick
behaveTrialIdx = vrexp.getBehaveTrialIdx(trialStartSample) # array of trial index for each sample
lickTrialIdx = behaveTrialIdx[lickSamples] # trial index of each lick
withinTrialSample = np.append(np.diff(behaveTrialIdx)==0, True) # only true if next sample is from same trial (last sample from each trial == False)

sampleDuration = np.append(np.diff(behaveTimeStamps),0) # time between samples, assume zero time was spent in last sample for each trial
behaveSpeed = np.append(np.diff(behavePosition)/sampleDuration[:-1], 0) # speed in each sample, assume speed was zero in last sample for each trial
sampleDurationThresholded = sampleDuration * (behaveSpeed >= speedThreshold) * withinTrialSample # keep sample duration when speed above threshold and sample within trial
behaveSpeedThresholded = behaveSpeed * (behaveSpeed >= speedThreshold) * withinTrialSample # keep speed when above threshold and sample within trial

# Get high resolution occupancy and speed maps 
occmap = np.zeros((vrexp.value['numTrials'],numPosition))
speedmap = np.zeros((vrexp.value['numTrials'],numPosition))
lickmap = np.zeros((vrexp.value['numTrials'],numPosition))
count = np.zeros((vrexp.value['numTrials'],numPosition))
functions.getMaps(sampleDurationThresholded, behaveSpeedThresholded, behaveTrialIdx, behavePositionBin, occmap, speedmap, count)
functions.getMap(np.ones_like(lickSamples), lickTrialIdx, lickPositionBin, lickmap)

# Figure out the valid range (outside of this range, set the maps to nan, because their values are not meaningful)
bpbPerTrial = vrexp.groupBehaveByTrial(behavePositionBin, trialStartSample)

# offsetting by 1 because there is a bug in the vrControl software where the first sample is always set 
# to the minimum position (which is 0), but if there is a built-up buffer in the rotary encoder, the position
# will jump at the second sample. In general this will always work unless the mice have a truly ridiculous
# speed at the beginning of the trial...
firstValidBin = [np.min(bpb[1:] if len(bpb)>1 else bpb) for bpb in bpbPerTrial] 
lastValidBin = [np.max(bpb) for bpb in bpbPerTrial]

In [153]:
lastValidBin[-1]

0

In [152]:
firstValidBin[-1]

0

In [149]:
bpbPerTrial[-1]

array([0], dtype=int64)

In [148]:
[len(bpb) for bpb in bpbPerTrial]

[102,
 101,
 94,
 93,
 96,
 115,
 110,
 117,
 107,
 121,
 151,
 125,
 126,
 135,
 127,
 132,
 113,
 116,
 131,
 117,
 117,
 119,
 115,
 115,
 129,
 116,
 119,
 118,
 111,
 149,
 106,
 127,
 122,
 107,
 131,
 128,
 189,
 128,
 126,
 118,
 127,
 127,
 126,
 124,
 131,
 129,
 125,
 104,
 120,
 115,
 107,
 111,
 125,
 140,
 122,
 115,
 124,
 107,
 116,
 111,
 124,
 136,
 132,
 112,
 119,
 135,
 115,
 142,
 114,
 113,
 120,
 118,
 140,
 114,
 107,
 110,
 132,
 129,
 113,
 106,
 116,
 147,
 121,
 112,
 112,
 114,
 125,
 132,
 134,
 111,
 116,
 132,
 115,
 106,
 152,
 112,
 118,
 123,
 124,
 144,
 114,
 104,
 102,
 127,
 135,
 132,
 135,
 100,
 121,
 109,
 135,
 134,
 172,
 123,
 196,
 108,
 104,
 122,
 113,
 109,
 199,
 137,
 217,
 116,
 103,
 94,
 132,
 111,
 95,
 112,
 123,
 125,
 117,
 112,
 104,
 124,
 108,
 121,
 115,
 125,
 124,
 304,
 691,
 363,
 891,
 318,
 210,
 162,
 111,
 99,
 108,
 104,
 101,
 104,
 111,
 107,
 109,
 104,
 120,
 108,
 109,
 110,
 112,
 110,
 95,
 108,
 113,
 114,

In [ ]:
# next step in analysis: 
# Also - using a cutoff point by the hitch in that distribution might ignore true matches that are soma-dendritic, but maybe I can use ROICaT or npix to avoid those? 

In [ ]:
# okay, looking at these clusters, it seems like there is some serious oversplitting going on in suite2p. That sucks!
# I think I should probably try to rerun some suite2p sessions....
# But for now I'll probably do a "merge" procedure... and I honestly think I'll have to overwrite or add a new "modified" suite2p folder with the stat and ops files